# **Semana 14: Feature Engineering y Optimización de Hiperparámetros**

## Ciencia de Datos en el Deporte - Análisis Avanzado

### **Bloque 3: Análisis Avanzado y Modelado**

---

**Objetivos de Aprendizaje:**
- Dominar técnicas de Feature Engineering para datos deportivos
- Crear variables predictivas más informativas
- Implementar Grid Search y Random Search
- Optimizar hiperparámetros sistemáticamente
- Mejorar el rendimiento de modelos existentes

**Herramientas:**
- Feature Engineering (variables sintéticas, transformaciones)
- GridSearchCV y RandomizedSearchCV
- Validación cruzada avanzada
- Análisis de importancia de características
- Técnicas de selección de variables

---

## **1. Configuración Inicial**

En esta semana daremos el siguiente paso: no solo evaluaremos qué tan buenos son nuestros modelos, sino que aprenderemos a **mejorarlos** creando mejores variables (Feature Engineering) y ajustando sus parámetros internos (Optimización de Hiperparámetros).

### **Analogía Deportiva**
Piensa en un entrenador de fútbol que no solo evalúa cómo juega su equipo, sino que:
- **Feature Engineering** = Crear nuevas jugadas y formaciones (nuevas variables a partir de las existentes)
- **Optimización** = Ajustar la alineación y estrategia para cada rival específico

In [1]:
# Importaciones básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Bibliotecas importadas correctamente")
print("Listo para Feature Engineering y Optimización")

Bibliotecas importadas correctamente
Listo para Feature Engineering y Optimización


In [2]:
# Cargar los datos deportivos
np.random.seed(42)

# Generar dataset deportivo ampliado para Feature Engineering
n_samples = 1000

data = {
    'altura': np.random.normal(175, 10, n_samples),
    'peso': np.random.normal(70, 12, n_samples),
    'edad': np.random.randint(18, 35, n_samples),
    'experiencia': np.random.randint(0, 15, n_samples),
    'velocidad_max': np.random.normal(25, 5, n_samples),
    'resistencia': np.random.normal(80, 15, n_samples),
    'precision_tiros': np.random.normal(65, 20, n_samples),
    'partidos_jugados': np.random.randint(10, 200, n_samples),
    'goles_temporada': np.random.randint(0, 30, n_samples),
    'asistencias': np.random.randint(0, 15, n_samples),
    'lesiones_historicas': np.random.randint(0, 5, n_samples)
}

# Crear DataFrame
df = pd.DataFrame(data)

# Variable objetivo: rendimiento del jugador (alta/media/baja)
# Basado en una combinación de variables
rendimiento_score = (
    (df['goles_temporada'] / 30) * 0.3 +
    (df['asistencias'] / 15) * 0.2 +
    (df['precision_tiros'] / 100) * 0.2 +
    (df['velocidad_max'] / 35) * 0.15 +
    (df['resistencia'] / 100) * 0.15
)

# Crear categorías de rendimiento
df['rendimiento'] = pd.cut(rendimiento_score, 
                          bins=[0, 0.4, 0.7, 1.0], 
                          labels=['Bajo', 'Medio', 'Alto'])

print(f"Dataset cargado: {df.shape[0]} jugadores, {df.shape[1]} variables")
print(f"Distribución de rendimiento:")
print(df['rendimiento'].value_counts())
print("\n" + "="*50)
df.head()

Dataset cargado: 1000 jugadores, 12 variables
Distribución de rendimiento:
rendimiento
Medio    759
Alto     203
Bajo      38
Name: count, dtype: int64



,altura,peso,edad,experiencia,velocidad_max,resistencia,precision_tiros,partidos_jugados,goles_temporada,asistencias,lesiones_historicas,rendimiento
0,179.967142,86.792265,23,9,26.087317,93.444259,16.514994,134,15,1,2,Medio
1,173.617357,81.095604,28,13,20.780496,80.317522,57.430907,193,19,2,0,Medio
2,181.476885,70.715564,20,12,16.653553,79.009175,71.053960,181,14,0,4,Medio
3,190.230299,62.236759,25,2,25.799383,102.437674,69.053567,194,13,14,4,Alto
4,172.658466,78.378680,20,13,20.669022,94.591616,102.998793,143,22,9,2,Alto


## **2. Feature Engineering - Creando Variables Inteligentes**

### **¿Qué es Feature Engineering?**

El **Feature Engineering** es el arte de crear nuevas variables más informativas a partir de las existentes. Es como ser un chef que combina ingredientes básicos para crear platos extraordinarios.

### **Analogía Deportiva: El Scouting Inteligente**

Imagina que eres un scout de fútbol:
- **Variables básicas**: altura, peso, velocidad
- **Variables engineered**: 
  - Índice Masa Corporal (BMI) = peso/altura²
  - Eficiencia de gol = goles/partidos_jugados
  - Experiencia relativa = experiencia/edad
  - Ratio físico = velocidad/peso

### **Tipos de Feature Engineering**

1. **Ratios e Índices**: Combinar variables existentes
2. **Transformaciones Matemáticas**: log, sqrt, potencias
3. **Binning**: Convertir numéricas en categorías
4. **Interacciones**: Productos entre variables
5. **Variables Polinómicas**: x², x³, etc.

In [3]:
# 1. RATIOS E ÍNDICES DEPORTIVOS
print("Creando Variables Engineered - Ratios e Índices")
print("="*50)

# Crear una copia para trabajar
df_featured = df.copy()

# Índice de Masa Corporal (BMI)
df_featured['bmi'] = df_featured['peso'] / (df_featured['altura']/100)**2

# Eficiencia de goles (goles por partido)
df_featured['eficiencia_gol'] = df_featured['goles_temporada'] / df_featured['partidos_jugados']

# Eficiencia de asistencias
df_featured['eficiencia_asistencia'] = df_featured['asistencias'] / df_featured['partidos_jugados']

# Productividad total (goles + asistencias por partido)
df_featured['productividad_total'] = (df_featured['goles_temporada'] + df_featured['asistencias']) / df_featured['partidos_jugados']

# Experiencia relativa (años de experiencia por edad)
df_featured['experiencia_relativa'] = df_featured['experiencia'] / df_featured['edad']

# Ratio físico (velocidad por peso - velocidad específica)
df_featured['ratio_velocidad_peso'] = df_featured['velocidad_max'] / df_featured['peso']

# Índice de robustez (resistencia vs lesiones)
df_featured['robustez'] = df_featured['resistencia'] / (df_featured['lesiones_historicas'] + 1)  # +1 para evitar división por 0

print("Variables creadas:")
nuevas_vars = ['bmi', 'eficiencia_gol', 'eficiencia_asistencia', 'productividad_total', 
               'experiencia_relativa', 'ratio_velocidad_peso', 'robustez']

for var in nuevas_vars:
    print(f"   • {var}: {df_featured[var].mean():.3f} (promedio)")

print(f"\nDataset ampliado: {df_featured.shape[1]} variables totales")
df_featured[nuevas_vars].head()

Creando Variables Engineered - Ratios e Índices
Variables creadas:
   • bmi: 23.321 (promedio)
   • eficiencia_gol: 0.236 (promedio)
   • eficiencia_asistencia: 0.113 (promedio)
   • productividad_total: 0.349 (promedio)
   • experiencia_relativa: 0.287 (promedio)
   • ratio_velocidad_peso: 0.364 (promedio)
   • robustez: 36.240 (promedio)

Dataset ampliado: 19 variables totales


,bmi,eficiencia_gol,eficiencia_asistencia,productividad_total,experiencia_relativa,ratio_velocidad_peso,robustez
0,26.797519,0.111940,0.007463,0.119403,0.391304,0.300572,31.148086
1,26.903639,0.098446,0.010363,0.108808,0.464286,0.256247,80.317522
2,21.471994,0.077348,0.000000,0.077348,0.600000,0.235501,15.801835
3,17.198382,0.067010,0.072165,0.139175,0.080000,0.414536,20.487535
4,26.291913,0.153846,0.062937,0.216783,0.650000,0.263707,31.530539


In [4]:
# 2. TRANSFORMACIONES MATEMÁTICAS
print("\nAplicando Transformaciones Matemáticas")
print("="*50)

# Transformaciones logarítmicas (útiles para variables asimétricas)
df_featured['log_experiencia'] = np.log1p(df_featured['experiencia'])  # log1p para manejar 0s
df_featured['log_partidos'] = np.log1p(df_featured['partidos_jugados'])

# Transformaciones de raíz cuadrada
df_featured['sqrt_resistencia'] = np.sqrt(df_featured['resistencia'])
df_featured['sqrt_precision'] = np.sqrt(np.abs(df_featured['precision_tiros']))  # abs para evitar negativos

# Variables al cuadrado (capturar relaciones no lineales)
df_featured['velocidad_cuadrada'] = df_featured['velocidad_max'] ** 2
df_featured['edad_cuadrada'] = df_featured['edad'] ** 2

print("Transformaciones aplicadas:")
transformaciones = ['log_experiencia', 'log_partidos', 'sqrt_resistencia', 
                   'sqrt_precision', 'velocidad_cuadrada', 'edad_cuadrada']

for trans in transformaciones:
    print(f"   • {trans}: rango [{df_featured[trans].min():.2f}, {df_featured[trans].max():.2f}]")

# 3. VARIABLES POLINÓMICAS AUTOMÁTICAS
print("\nCreando Interacciones Polinómicas")
print("="*50)

# Seleccionar variables clave para interacciones
vars_clave = ['altura', 'peso', 'velocidad_max', 'resistencia', 'precision_tiros']

# Crear características polinómicas (grado 2 = incluye interacciones)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly_features = poly.fit_transform(df_featured[vars_clave])

# Obtener nombres de las nuevas características
poly_names = poly.get_feature_names_out(vars_clave)

# Agregar solo las interacciones (no las variables originales)
interaction_features = poly_features[:, len(vars_clave):]  # Omitir las primeras columnas originales
interaction_names = poly_names[len(vars_clave):]

# Crear DataFrame con interacciones
df_interactions = pd.DataFrame(interaction_features, columns=interaction_names)

# Agregar las más importantes (primeras 5 para mantener manageable)
for i, col in enumerate(interaction_names[:5]):
    clean_name = col.replace(' ', '_').replace('*', '_x_')
    df_featured[f'interact_{clean_name}'] = df_interactions.iloc[:, i]

print(f"{len(interaction_names[:5])} interacciones principales creadas")
print(f"Dataset final: {df_featured.shape[1]} variables totales")

# Mostrar estadísticas finales
print(f"\nResumen del Feature Engineering:")
print(f"   • Variables originales: {len(df.columns)}")
print(f"   • Variables engineered: {df_featured.shape[1] - len(df.columns)}")
print(f"   • Total final: {df_featured.shape[1]}")


Aplicando Transformaciones Matemáticas
Transformaciones aplicadas:
   • log_experiencia: rango [0.00, 2.71]
   • log_partidos: rango [2.40, 5.30]
   • sqrt_resistencia: rango [4.94, 11.02]
   • sqrt_precision: rango [1.93, 11.70]
   • velocidad_cuadrada: rango [77.49, 1873.90]
   • edad_cuadrada: rango [324.00, 1156.00]

Creando Interacciones Polinómicas
5 interacciones principales creadas
Dataset final: 30 variables totales

Resumen del Feature Engineering:
   • Variables originales: 12
   • Variables engineered: 18
   • Total final: 30


## **3. Selección de Características - Identificando las Variables Estrella**

### **¿Por qué seleccionar características?**

No todas las variables creadas serán útiles. Algunas pueden:
- **Generar ruido** en lugar de información
- **Ralentizar** el entrenamiento 
- **Correlacionarse** demasiado con otras (redundancia)

### **Técnicas de Selección**

1. **Análisis Univariado**: F-score, correlación
2. **Selección Recursiva**: RFE (Recursive Feature Elimination)
3. **Importancia de Modelo**: Random Forest feature importance
4. **Análisis de Correlación**: Detectar redundancia

In [5]:
# PREPARAR DATOS PARA SELECCIÓN
print("Selección de Características - Identificando Variables Estrella")
print("="*60)

# Preparar X e y (convertir categórica a numérica)
X = df_featured.drop('rendimiento', axis=1)
y = df_featured['rendimiento'].map({'Bajo': 0, 'Medio': 1, 'Alto': 2})

# Manejar valores infinitos o NaN
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.mean())

print(f"Analizando {X.shape[1]} características para predecir rendimiento")

# 1. ANÁLISIS UNIVARIADO con F-score
print("\n1. Análisis Univariado (F-score)")
print("-" * 40)

selector_f = SelectKBest(score_func=f_classif, k=10)
X_f_selected = selector_f.fit_transform(X, y)

# Obtener puntuaciones y características seleccionadas
f_scores = selector_f.scores_
selected_features_f = X.columns[selector_f.get_support()]

print("Top 10 características por F-score:")
feature_scores = list(zip(X.columns, f_scores))
feature_scores.sort(key=lambda x: x[1], reverse=True)

for i, (feature, score) in enumerate(feature_scores[:10]):
    print(f"   {i+1:2d}. {feature:<25} | F-score: {score:.2f}")

# 2. IMPORTANCIA CON RANDOM FOREST
print("\n2. Importancia con Random Forest")
print("-" * 40)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Obtener importancias
importances = rf.feature_importances_
feature_importance = list(zip(X.columns, importances))
feature_importance.sort(key=lambda x: x[1], reverse=True)

print("Top 10 características por importancia RF:")
for i, (feature, importance) in enumerate(feature_importance[:10]):
    print(f"   {i+1:2d}. {feature:<25} | Importancia: {importance:.4f}")

# 3. SELECCIÓN RECURSIVA (RFE)
print("\n3. Selección Recursiva (RFE)")
print("-" * 40)

rfe = RFE(estimator=LogisticRegression(random_state=42, max_iter=1000), n_features_to_select=10)
X_rfe_selected = rfe.fit_transform(X, y)
selected_features_rfe = X.columns[rfe.support_]

print("Top 10 características por RFE:")
for i, feature in enumerate(selected_features_rfe):
    ranking = rfe.ranking_[X.columns.get_loc(feature)]
    print(f"   {i+1:2d}. {feature:<25} | Ranking: {ranking}")

print(f"\nResumen de Selección:")
print(f"   • Método F-score: {len(selected_features_f)} características")
print(f"   • Método RF: Top 10 por importancia")
print(f"   • Método RFE: {len(selected_features_rfe)} características")

Selección de Características - Identificando Variables Estrella
Analizando 29 características para predecir rendimiento

1. Análisis Univariado (F-score)
----------------------------------------
Top 10 características por F-score:
    1. goles_temporada           | F-score: 221.75
    2. asistencias               | F-score: 116.34
    3. precision_tiros           | F-score: 60.45
    4. interact_altura_precision_tiros | F-score: 58.99
    5. sqrt_precision            | F-score: 56.50
    6. productividad_total       | F-score: 38.79
    7. eficiencia_gol            | F-score: 36.12
    8. eficiencia_asistencia     | F-score: 23.57
    9. velocidad_cuadrada        | F-score: 12.72
   10. interact_altura_velocidad_max | F-score: 12.51

2. Importancia con Random Forest
----------------------------------------
Top 10 características por importancia RF:
    1. goles_temporada           | Importancia: 0.1832
    2. asistencias               | Importancia: 0.1053
    3. productividad_total   

## **4. Optimización de Hiperparámetros - Ajustando la Máquina**

### **¿Qué son los Hiperparámetros?**

Los **hiperparámetros** son los "ajustes" del modelo que NO se aprenden de los datos, sino que nosotros debemos configurar antes del entrenamiento.

### **Analogía Deportiva: Ajustando un Coche de Carreras**

Imagina que eres mecánico de F1:
- **Motor (algoritmo)**: Random Forest, SVM, etc.
- **Hiperparámetros**: 
  - Presión de neumáticos (learning rate)
  - Aerodinámica (regularización)
  - Configuración del motor (n_estimators, max_depth)

### **Técnicas de Optimización**

1. **Grid Search**: Prueba TODAS las combinaciones (exhaustivo pero lento)
2. **Random Search**: Prueba combinaciones aleatorias (más eficiente)
3. **Bayesian Optimization**: Usa experiencia previa (más inteligente)

In [6]:
# PREPARAR DATOS PARA OPTIMIZACIÓN
print("Optimización de Hiperparámetros - Ajustando la Máquina")
print("="*60)

# Usar las mejores características identificadas (top 10 por F-score)
X_best = X[selected_features_f]
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42, stratify=y)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Usando {X_best.shape[1]} mejores características")
print(f"Datos: {X_train.shape[0]} entrenamiento, {X_test.shape[0]} prueba")

# 1. GRID SEARCH para Random Forest
print("\n1. Grid Search - Random Forest")
print("-" * 40)

# Definir parámetros a probar
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear Grid Search
grid_search_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,  # 5-fold cross validation
    scoring='accuracy',
    n_jobs=-1,  # Usar todos los procesadores
    verbose=1
)

print(f"Probando {len(param_grid_rf['n_estimators']) * len(param_grid_rf['max_depth']) * len(param_grid_rf['min_samples_split']) * len(param_grid_rf['min_samples_leaf'])} combinaciones...")

# Ejecutar búsqueda
import time
start_time = time.time()
grid_search_rf.fit(X_train_scaled, y_train)
grid_time = time.time() - start_time

print(f"Grid Search completado en {grid_time:.1f} segundos")
print(f"Mejor puntuación CV: {grid_search_rf.best_score_:.4f}")
print(f"Mejores parámetros:")
for param, value in grid_search_rf.best_params_.items():
    print(f"   • {param}: {value}")

# Evaluar en test
best_rf = grid_search_rf.best_estimator_
rf_test_score = best_rf.score(X_test_scaled, y_test)
print(f"Accuracy en test: {rf_test_score:.4f}")

Optimización de Hiperparámetros - Ajustando la Máquina
Usando 10 mejores características
Datos: 800 entrenamiento, 200 prueba

1. Grid Search - Random Forest
----------------------------------------
Probando 108 combinaciones...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Grid Search completado en 18.6 segundos
Mejor puntuación CV: 0.9163
Mejores parámetros:
   • max_depth: 10
   • min_samples_leaf: 2
   • min_samples_split: 5
   • n_estimators: 100
Accuracy en test: 0.9150
Grid Search completado en 18.6 segundos
Mejor puntuación CV: 0.9163
Mejores parámetros:
   • max_depth: 10
   • min_samples_leaf: 2
   • min_samples_split: 5
   • n_estimators: 100
Accuracy en test: 0.9150


In [9]:
# 2. RANDOM SEARCH para SVM
print("\n2. Random Search - SVM")
print("-" * 40)

# Definir distribuciones de parámetros para SVM
from scipy.stats import uniform, randint

param_dist_svm = {
    'C': uniform(0.1, 10),  # Entre 0.1 y 10.1
    'gamma': uniform(0.001, 0.1),  # Entre 0.001 y 0.101
    'kernel': ['rbf', 'poly', 'sigmoid']
}

# Crear Random Search
random_search_svm = RandomizedSearchCV(
    estimator=SVC(random_state=42),
    param_distributions=param_dist_svm,
    n_iter=50,  # Número de combinaciones aleatorias a probar
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print(f"Probando 50 combinaciones aleatorias...")

start_time = time.time()
random_search_svm.fit(X_train_scaled, y_train)
random_time = time.time() - start_time

print(f"Random Search completado en {random_time:.1f} segundos")
print(f"Mejor puntuación CV: {random_search_svm.best_score_:.4f}")
print(f"Mejores parámetros:")
for param, value in random_search_svm.best_params_.items():
    if isinstance(value, float):
        print(f"   • {param}: {value:.4f}")
    else:
        print(f"   • {param}: {value}")

# Evaluar en test
best_svm = random_search_svm.best_estimator_
svm_test_score = best_svm.score(X_test_scaled, y_test)
print(f"Accuracy en test: {svm_test_score:.4f}")

# 3. COMPARACIÓN DE EFICIENCIA
print("\n3. Comparación de Eficiencia")
print("-" * 40)
print(f"Grid Search (RF):   {grid_time:.1f}s para 108 combinaciones")
print(f"Random Search (SVM): {random_time:.1f}s para 50 combinaciones")
print(f"RF Test Accuracy: {rf_test_score:.4f}")
print(f"SVM Test Accuracy: {svm_test_score:.4f}")

if rf_test_score > svm_test_score:
    winner = "Random Forest (Grid Search)"
    diff = rf_test_score - svm_test_score
else:
    winner = "SVM (Random Search)"
    diff = svm_test_score - rf_test_score

print(f"\n{winner} gana por {diff:.4f} puntos!")


2. Random Search - SVM
----------------------------------------
Probando 50 combinaciones aleatorias...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Random Search completado en 0.4 segundos
Mejor puntuación CV: 0.9425
Mejores parámetros:
   • C: 6.3435
   • gamma: 0.0306
   • kernel: rbf
Accuracy en test: 0.9250

3. Comparación de Eficiencia
----------------------------------------
Grid Search (RF):   18.6s para 108 combinaciones
Random Search (SVM): 0.4s para 50 combinaciones
RF Test Accuracy: 0.9150
SVM Test Accuracy: 0.9250

SVM (Random Search) gana por 0.0100 puntos!
Random Search completado en 0.4 segundos
Mejor puntuación CV: 0.9425
Mejores parámetros:
   • C: 6.3435
   • gamma: 0.0306
   • kernel: rbf
Accuracy en test: 0.9250

3. Comparación de Eficiencia
----------------------------------------
Grid Search (RF):   18.6s para 108 combinaciones
Random Search (SVM): 0.4s para 50 combinaciones
RF Test Accuracy: 0.9150
SVM Test Accuracy: 0.9250

SVM (Random Search)

## **5. Interpretación Práctica - De los Números a las Decisiones**

### **¿Qué aprendimos hoy?**

#### **Feature Engineering Efectivo**
- **Ratios deportivos** (eficiencia de gol, BMI) son más informativos que variables brutas
- **Interacciones** entre altura y precisión revelan patrones ocultos
- **Transformaciones** matemáticas capturan relaciones no lineales

#### **Optimización Inteligente**
- **Random Search** puede ser más eficiente que Grid Search
- **50 combinaciones aleatorias** vs **108 sistemáticas**
- **SVM optimizado** superó a Random Forest por 1%

### **Aplicaciones Deportivas Reales**

#### **Para un Scout de Fútbol**
```python
# Variables más importantes identificadas:
top_variables = [
    'goles_temporada',      # Productividad ofensiva
    'asistencias',          # Visión de juego  
    'precision_tiros',      # Calidad técnica
    'productividad_total',  # Rendimiento integral
    'eficiencia_gol'        # Consistencia
]
```

#### **Recomendaciones Prácticas**
1. **Crear ratios** siempre que tengas métricas base (goles/partidos)
2. **Probar interacciones** entre características físicas y técnicas
3. **Usar Random Search** para optimización rápida inicial
4. **Validar SIEMPRE** en datos de prueba independientes

In [8]:
# RESUMEN EJECUTIVO FINAL
print("RESUMEN EJECUTIVO - Feature Engineering y Optimización")
print("="*65)

# Comparar modelo base vs optimizado
print("COMPARACIÓN DE RENDIMIENTO:")
print("-" * 40)

# Modelo base (sin feature engineering)
X_original = df[['altura', 'peso', 'velocidad_max', 'resistencia', 'precision_tiros']]
X_orig_train, X_orig_test, y_orig_train, y_orig_test = train_test_split(X_original, y, test_size=0.2, random_state=42, stratify=y)

# Escalar datos originales
X_orig_train_scaled = scaler.fit_transform(X_orig_train)
X_orig_test_scaled = scaler.transform(X_orig_test)

# Entrenar modelo base
base_rf = RandomForestClassifier(n_estimators=100, random_state=42)
base_rf.fit(X_orig_train_scaled, y_orig_train)
base_score = base_rf.score(X_orig_test_scaled, y_orig_test)

# Mejora obtenida
improvement = svm_test_score - base_score

print(f"Modelo Base (5 variables originales):     {base_score:.4f}")
print(f"Modelo Optimizado (10 features + tuning): {svm_test_score:.4f}")
print(f"Mejora obtenida: +{improvement:.4f} ({improvement*100:.1f}%)")

print(f"\nESTRATEGIA GANADORA:")
print(f"   1. Feature Engineering: +{df_featured.shape[1] - df.shape[1]} variables sintéticas")
print(f"   2. Selección: {len(selected_features_f)} mejores características")
print(f"   3. Optimización: Random Search en {random_time:.1f}s")
print(f"   4. Resultado: 92.5% accuracy")

print(f"\nLECCIONES CLAVE:")
print(f"   • Los ratios (eficiencia_gol, productividad) son MUY informativos")
print(f"   • Random Search es {grid_time/random_time:.1f}x más rápido que Grid Search")
print(f"   • Feature Engineering puede mejorar {improvement*100:.1f}% el rendimiento")
print(f"   • SIEMPRE validar en datos independientes")

print(f"\nPRÓXIMOS PASOS RECOMENDADOS:")
print(f"   1. Probar Bayesian Optimization (más inteligente)")
print(f"   2. Explorar más interacciones específicas del deporte")
print(f"   3. Implementar en pipeline de producción")
print(f"   4. Validar con datos de temporadas diferentes")

print("\n" + "="*65)
print("¡FELICITACIONES! Dominas Feature Engineering y Optimización")
print("="*65)

RESUMEN EJECUTIVO - Feature Engineering y Optimización
COMPARACIÓN DE RENDIMIENTO:
----------------------------------------
Modelo Base (5 variables originales):     0.7850
Modelo Optimizado (10 features + tuning): 0.9250
Mejora obtenida: +0.1400 (14.0%)

ESTRATEGIA GANADORA:
   1. Feature Engineering: +18 variables sintéticas
   2. Selección: 10 mejores características
   3. Optimización: Random Search en 0.5s
   4. Resultado: 92.5% accuracy

LECCIONES CLAVE:
   • Los ratios (eficiencia_gol, productividad) son MUY informativos
   • Random Search es 40.6x más rápido que Grid Search
   • Feature Engineering puede mejorar 14.0% el rendimiento
   • SIEMPRE validar en datos independientes

PRÓXIMOS PASOS RECOMENDADOS:
   1. Probar Bayesian Optimization (más inteligente)
   2. Explorar más interacciones específicas del deporte
   3. Implementar en pipeline de producción
   4. Validar con datos de temporadas diferentes

¡FELICITACIONES! Dominas Feature Engineering y Optimización
